In [ ]:
import os
import time
from picamera2 import Picamera2
import cv2
import numpy as np
import onnxruntime as ort

SNAPSHOT_DIR = "/home/aman1234/trash_snapshots"
os.makedirs(SNAPSHOT_DIR, exist_ok=True)

ort_session = ort.InferenceSession("/home/aman1234/Downloads/best.onnx")

def preprocess(frame):
    """Resize, normalize, convert to NCHW."""
    img = cv2.resize(frame, (640, 640))
    img = img.astype(np.float32) / 255.0
    img = np.transpose(img, (2, 0, 1))
    img = np.expand_dims(img, axis=0)
    return img

def postprocess(outputs, conf_threshold=0.5):  # Threshold updated to 0.5
    preds = outputs[0][0]  # shape (5, 8400)
    boxes, scores = [], []

    for i in range(preds.shape[1]):
        x_c, y_c, w, h, conf = preds[:, i]
        if conf < conf_threshold:
            continue

        # Coordinates are absolute pixels
        x1 = int(x_c - w / 2)
        x2 = int(x_c + w / 2)
        y1 = int(y_c - h / 2)
        y2 = int(y_c + h / 2)

        # Clip to frame size (640x480)
        x1, x2 = max(0, x1), min(639, x2)
        y1, y2 = max(0, y1), min(479, y2)

        if x1 >= x2 or y1 >= y2:
            continue

        boxes.append([x1, y1, x2, y2])
        scores.append(conf)

    return boxes, scores

def draw_boxes(frame, boxes, scores):
    """Draw boxes and confidence scores (small) on frame."""
    for box, score in zip(boxes, scores):
        x1, y1, x2, y2 = box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        conf_text = f"{score:.2f}"
        y_text = y1 - 5 if y1 - 5 > 10 else y1 + 15
        cv2.putText(frame, conf_text, (x1, y_text),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

def main():
    picam2 = Picamera2()
    config = picam2.create_preview_configuration(main={"format": "RGB888", "size": (640, 480)})
    picam2.configure(config)
    picam2.start()

    last_no_trash_msg = 0
    last_trash_msg = 0
    last_snapshot = 0

    try:
        while True:
            frame = picam2.capture_array()  # RGB frame

            input_tensor = preprocess(frame)
            outputs = ort_session.run(None, {ort_session.get_inputs()[0].name: input_tensor})

            boxes, scores = postprocess(outputs)

            # Convert RGB to BGR for OpenCV drawing and display
            frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

            if boxes:
                draw_boxes(frame_bgr, boxes, scores)

                if time.time() - last_trash_msg > 3:
                    print("Trash detected")
                    last_trash_msg = time.time()

                if time.time() - last_snapshot > 5:
                    timestamp = time.strftime("%Y%m%d_%H%M%S")
                    snapshot_path = os.path.join(SNAPSHOT_DIR, f"trash_{timestamp}.jpg")
                    cv2.imwrite(snapshot_path, frame_bgr)
                    print(f"Saved snapshot: {snapshot_path}")
                    last_snapshot = time.time()

                cv2.putText(frame_bgr, "Trash", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                if time.time() - last_no_trash_msg > 10:
                    print("No trash detected")
                    last_no_trash_msg = time.time()

                cv2.putText(frame_bgr, "No Trash", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            cv2.imshow("Trash Detection", frame_bgr)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except KeyboardInterrupt:
        print("Exiting...")

    finally:
        picam2.stop()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
